In [ ]:
##### ENVIRONMENT SETUP ########

import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import scipy.stats as stats
from scipy.stats import norm
import lightgbm as lgb

pd.options.display.max_columns = 999

##### TRAINING DATA ############

train_data = pd.read_csv('D:/AI ML/train_2016_v2.csv', parse_dates =["transactiondate"])
# print train_data.shape
train_data.head(10)

##### PROPERTY DATA ############

property_data = pd.read_csv('D:/AI ML/2016DATA.csv')
# print property_data.shape
property_data.head(10)
# print property_data.dtypes

##### Visualize and Clean Data #####

plt.figure(figsize=(12,20))
property_data.drop('parcelid',axis=1).notnull().mean().sort_values(ascending = True).plot(kind = 'barh')
plt.title('Percentage of Information by Feature: (2016 Data)')
plt.show()
property_data.drop('parcelid',axis=1).notnull().mean().sort_values(ascending = False)

##### Pools & Hot tubs ##########

# "0 pools"
property_data.poolcnt.fillna(0,inplace = True)
# "0 hot tubs or spas"
property_data.hashottuborspa.fillna(0,inplace = True)
# Convert "True" to 1
property_data.hashottuborspa.replace(to_replace = True, value = 1,inplace = True)
# print property_data['hashottuborspa'].value_counts()

# Set properties that have a pool but no info on poolsize equal to the median poolsize value.
property_data.loc[property_data.poolcnt==1, 'poolsizesum'] = property_data.loc[property_data.poolcnt==1, 'poolsizesum'].fillna(property_data[property_data.poolcnt==1].poolsizesum.median())
# "0 pools" = "0 sq ft of pools"
property_data.loc[property_data.poolcnt==0, 'poolsizesum']=0
# "0 pools with a spa/hot tub"
property_data.pooltypeid2.fillna(0,inplace = True)
# "0 pools without a hot tub"
property_data.pooltypeid7.fillna(0,inplace = True)
# Drop redundant feature
property_data.drop('pooltypeid10', axis=1, inplace=True)
property_data.head()

##### Fireplace Data ##########

# print property_data['fireplaceflag'].value_counts()
# If "fireplaceflag" is "True" and "fireplacecnt" is "NaN", we will set "fireplacecnt" equal to the median value of "1".
property_data.loc[(property_data['fireplaceflag'] == True) & (property_data['fireplacecnt'].isnull()), ['fireplacecnt']] = 1
# If 'fireplacecnt' is "NaN", replace with "0"
property_data.fireplacecnt.fillna(0,inplace = True)
# If "fireplacecnt" is 1 or larger "fireplaceflag" is "NaN", we will set "fireplaceflag" to "True".
property_data.loc[(property_data['fireplacecnt'] >= 1.0) & (property_data['fireplaceflag'].isnull()), ['fireplaceflag']] = True
property_data.fireplaceflag.fillna(0,inplace = True)
# print property_data['fireplaceflag'].value_counts()
# Convert "True" to 1
property_data.fireplaceflag.replace(to_replace = True, value = 1,inplace = True)
# print property_data['fireplaceflag'].value_counts()

##### Garage Data ##########

property_data.garagecarcnt.fillna(0,inplace = True)
property_data.garagetotalsqft.fillna(0,inplace = True)

##### Tax Data Delinquency ##########

#print property_data['taxdelinquencyflag'].value_counts()
# Replace "NaN" with "0"
property_data.taxdelinquencyflag.fillna(0,inplace = True)
# Change "Y" to "1"
property_data.taxdelinquencyflag.replace(to_replace = 'Y', value = 1,inplace = True)
# print property_data['taxdelinquencyflag'].value_counts()
# Drop "taxdelinquencyyear"
property_data.drop('taxdelinquencyyear', axis=1, inplace=True)

##### The Rest ##########

# Drop "storytypeid"
property_data.drop('storytypeid', axis=1, inplace=True)
# Replace "NaN" with 0, signifying no basement.
property_data.basementsqft.fillna(0,inplace = True)
#print(property_data['yardbuildingsqft26'].value_counts())
# Replace 'yardbuildingsqft26' "NaN"s with "0".
property_data.yardbuildingsqft26.fillna(0,inplace = True)
# Drop "architecturalstyletypeid"
property_data.drop('architecturalstyletypeid', axis=1, inplace=True)
# Drop "typeconstructiontypeid" and "finishedsquarefeet13"
property_data.drop('typeconstructiontypeid', axis=1, inplace=True)
property_data.drop('finishedsquarefeet13', axis=1, inplace=True)
# Drop "buildingclasstypeid"
property_data.drop('buildingclasstypeid', axis=1, inplace=True)
#print property_data.shape
property_data.notnull().mean().sort_values(ascending = False)
# Let's check the unique values for "decktypeid"
# print(property_data['decktypeid'].value_counts())
# Change "decktypeid" "Nan"s to "0"
property_data.decktypeid.fillna(0,inplace = True)
# Convert "decktypeid" "66.0" to "1"
property_data.decktypeid.replace(to_replace = 66.0, value = 1,inplace = True)
# print(property_data['decktypeid'].value_counts())
# print(property_data['finishedsquarefeet6'].value_counts())

#squarefeet = property_data[property_data['finishedsquarefeet6'].notnull() & property_data['finishedsquarefeet12'].isnull() & property_data['finishedsquarefeet15'].isnull() & property_data['finishedsquarefeet50'].isnull() & property_data['lotsizesquarefeet'].isnull()]
#squarefeet = property_data[property_data['finishedsquarefeet12'].notnull() & property_data['finishedsquarefeet15'].notnull() & property_data['finishedsquarefeet50'].notnull() & property_data['lotsizesquarefeet'].notnull()]
squarefeet = property_data[property_data['finishedsquarefeet15'].notnull() & property_data['finishedsquarefeet50'].notnull() & property_data['lotsizesquarefeet'].notnull()]
squarefeet[['calculatedfinishedsquarefeet','finishedsquarefeet6','finishedsquarefeet12','finishedsquarefeet15','finishedsquarefeet50','numberofstories','lotsizesquarefeet','landtaxvaluedollarcnt','structuretaxvaluedollarcnt','taxvaluedollarcnt','taxamount']]
#squarefeet
# squarefeet = property_data[property_data[['finishedsquarefeet6','finishedsquarefeet12','finishedsquarefeet15','finishedsquarefeet50','lotsizesquarefeet']].notnull()]
#property_data[['finishedsquarefeet6','finishedsquarefeet12','finishedsquarefeet15','finishedsquarefeet50','lotsizesquarefeet']][:100]

# Drop "finishedsquarefeet6"
property_data.drop('finishedsquarefeet6', axis=1, inplace=True)
# Drop "finishedsquarefeet12"
property_data.drop('finishedsquarefeet12', axis=1, inplace=True)
# Drop "finishedfloor1squarefeet"
property_data.drop('finishedfloor1squarefeet', axis=1, inplace=True)

squarefeet2 = property_data[property_data['finishedsquarefeet15'].notnull() & property_data['finishedsquarefeet50'].notnull() & property_data['lotsizesquarefeet'].notnull()]
#squarefeet2 = property_data[property_data['finishedsquarefeet15'].notnull() & property_data['calculatedfinishedsquarefeet'].isnull()]
squarefeet2[['calculatedfinishedsquarefeet','finishedsquarefeet15','finishedsquarefeet50','numberofstories','lotsizesquarefeet']]
property_data.notnull().mean().sort_values(ascending = False)

# Replace "NaN" "calculatedfinishedsquarefeet" values with mean.
property_data['calculatedfinishedsquarefeet'].fillna((property_data['calculatedfinishedsquarefeet'].mean()), inplace=True)

# Replace "NaN" "finishedsquarefeet15" values with calculatedfinishedsquarefeet.
property_data.loc[property_data['finishedsquarefeet15'].isnull(),'finishedsquarefeet15'] = property_data['calculatedfinishedsquarefeet']
#property_data['finishedsquarefeet15'].fillna(property_data['calculatedfinishedsquarefeet'])

property_data.numberofstories.fillna(1,inplace = True)
property_data.notnull().mean().sort_values(ascending = False)
# print(property_data['numberofstories'].value_counts())

# If "numberofstories" is equal to "1", then we can replace the "NaN"s with the "calculatedfinishedsquarefeet" value. Fill in the rest with the average values.
property_data.loc[property_data['numberofstories'] == 1.0,'finishedsquarefeet50'] = property_data['calculatedfinishedsquarefeet']
property_data['finishedsquarefeet50'].fillna((property_data['finishedsquarefeet50'].mean()), inplace=True)

# print property_data.shape
property_data.notnull().mean().sort_values(ascending = False)

# Replace 'yardbuildingsqft17' "NaN"s with "0".
property_data.yardbuildingsqft17.fillna(0,inplace = True)

bathrooms = property_data[property_data['fullbathcnt'].notnull() & property_data['threequarterbathnbr'].notnull() & property_data['calculatedbathnbr'].notnull()]
bathrooms[['fullbathcnt','threequarterbathnbr','calculatedbathnbr']]

# Drop "threequarterbathnbr"
property_data.drop('threequarterbathnbr', axis=1, inplace=True)
# Drop "fullbathcnt"
property_data.drop('fullbathcnt', axis=1, inplace=True)
# Fill in "NaN" "calculatedbathnbr" with most common
bathroommode = property_data['calculatedbathnbr'].value_counts().idxmax()
property_data['calculatedbathnbr'] = property_data['calculatedbathnbr'].fillna(bathroommode)

# print property_data.shape
property_data.notnull().mean().sort_values(ascending = False)
property_data.airconditioningtypeid.fillna(5,inplace = True)

# Drop "regionidneighborhood"
property_data.drop('regionidneighborhood', axis=1, inplace=True)
property_data.heatingorsystemtypeid.fillna(13,inplace = True)
# print(property_data['buildingqualitytypeid'].value_counts())

# Fill in "NaN" "buildingqualitytypeid" with most common
buildingqual = property_data['buildingqualitytypeid'].value_counts().idxmax()
property_data['buildingqualitytypeid'] = property_data['buildingqualitytypeid'].fillna(buildingqual)

property_data.unitcnt.fillna(1,inplace = True)
# print(property_data['propertyzoningdesc'].value_counts())

# Fill in "NaN" "propertyzoningdesc" with most common
propertyzoningdesc = property_data['propertyzoningdesc'].value_counts().idxmax()
property_data['propertyzoningdesc'] = property_data['propertyzoningdesc'].fillna(propertyzoningdesc)
property_data['lotsizesquarefeet'].fillna((property_data['lotsizesquarefeet'].mean()), inplace=True)

# print(property_data['censustractandblock'].value_counts())
# print(property_data['rawcensustractandblock'].value_counts())

# Drop "censustractandblock"
property_data.drop('censustractandblock', axis=1, inplace=True)
property_data.landtaxvaluedollarcnt.fillna(0,inplace = True)
property_data.structuretaxvaluedollarcnt.fillna(0,inplace = True)
property_data['taxvaluedollarcnt'].fillna((property_data['taxvaluedollarcnt'].mean()), inplace=True)

property_data['taxpercentage'] = property_data['taxamount'] / property_data['taxvaluedollarcnt']
property_data.head()
property_data['taxpercentage'].fillna((property_data['taxpercentage'].mean()), inplace=True)

# Drop "taxamount"
property_data.drop('taxamount', axis=1, inplace=True)
# Drop "regionidcity"
property_data.drop('regionidcity', axis=1, inplace=True)

# Fill in "NaN" "yearbuilt" with most common
yearbuilt = property_data['yearbuilt'].value_counts().idxmax()
property_data['yearbuilt'] = property_data['yearbuilt'].fillna(yearbuilt)


#print property_data.shape
#print property_data.dtypes
property_data.notnull().mean().sort_values(ascending = False)

# Fill in "fips" "NaN"s
fips = property_data['fips'].value_counts().idxmax()
property_data['fips'] = property_data['fips'].fillna(fips)

# Fill in "propertylandusetypeid" "NaN"s
propertylandusetypeid = property_data['propertylandusetypeid'].value_counts().idxmax()
property_data['propertylandusetypeid'] = property_data['propertylandusetypeid'].fillna(propertylandusetypeid)

# Drop 'regionidcounty'
property_data.drop('regionidcounty', axis=1, inplace=True)

# Fill in "latitude" "NaN"s
latitude = property_data['latitude'].value_counts().idxmax()
property_data['latitude'] = property_data['latitude'].fillna(latitude)

# Fill in "longitude" "NaN"s
longitude = property_data['longitude'].value_counts().idxmax()
property_data['longitude'] = property_data['longitude'].fillna(longitude)


# Fill in "rawcensustractandblock" "NaN"s
rawcensustractandblock = property_data['rawcensustractandblock'].value_counts().idxmax()
property_data['rawcensustractandblock'] = property_data['rawcensustractandblock'].fillna(rawcensustractandblock)

# Fill in "assessmentyear" "NaN"s
assessmentyear = property_data['assessmentyear'].value_counts().idxmax()
property_data['assessmentyear'] = property_data['assessmentyear'].fillna(assessmentyear)

# Fill in "bedroomcnt" "NaN"s
bedroomcnt = property_data['bedroomcnt'].value_counts().idxmax()
property_data['bedroomcnt'] = property_data['bedroomcnt'].fillna(bedroomcnt)

# Fill in "bathroomcnt" "NaN"s
bathroomcnt = property_data['bathroomcnt'].value_counts().idxmax()
property_data['bathroomcnt'] = property_data['bathroomcnt'].fillna(bathroomcnt)

# Fill in "roomcnt" "NaN"s
roomcnt = property_data['roomcnt'].value_counts().idxmax()
property_data['roomcnt'] = property_data['roomcnt'].fillna(roomcnt)

# Fill in "propertycountylandusecode" "NaN"s
propertycountylandusecode = property_data['propertycountylandusecode'].value_counts().idxmax()
property_data['propertycountylandusecode'] = property_data['propertycountylandusecode'].fillna(propertycountylandusecode)

# Fill in "regionidzip " "NaN"s
regionidzip = property_data['regionidzip'].value_counts().idxmax()
property_data['regionidzip'] = property_data['regionidzip'].fillna(regionidzip)

#print property_data.shape
property_data.notnull().mean().sort_values(ascending = False)

###### LOG ERROR ###########

train_with_months = train_data
train_with_months['sale_month'] = train_with_months['transactiondate'].apply(lambda x: (x.to_pydatetime()).month)
train_with_months['sale_day'] = train_with_months['transactiondate'].apply(lambda x: (x.to_pydatetime()).day)
train_with_months['sale_year'] = train_with_months['transactiondate'].apply(lambda x: (x.to_pydatetime()).year)
train_with_months.head(10)

######### SALES PER MONTH #########

color = sns.color_palette()
num_of_sales = train_with_months['sale_month'].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(num_of_sales.index, num_of_sales.values, alpha=0.8, color=color[5])
plt.xticks(rotation='vertical')
plt.xlabel('Sale Month', fontsize=12)
plt.ylabel('Number of Sales', fontsize=12)
plt.show()

######### NORMAL LOG ERROR #########

plt.figure(figsize=(12,4))
sns.distplot(train_with_months.logerror.values, bins=500, kde=False)
plt.xlabel('Log Error')
mu, std = norm.fit(train_with_months.logerror.values)
xmin,xmax=plt.xlim()
x = np.linspace(-5,5,1000)
p = norm.pdf(x, 0, 0.025)
plt.plot(x, p*(x[1]-x[0])*90275, 'k', linewidth=2)
plt.axis([-0.3,0.3,0,20000])
plt.ylabel('Normal')
plt.show()

######### AVERAGE LOG ERROR PER MONTH #########

avg_logs_table = []
for x in range(1,13):
    avg_logs = np.mean(train_with_months[train_with_months['sale_month'] == x]['logerror'])
    avg_logs_table.append(avg_logs)

t = range(1,13)
plt.plot(t,avg_logs_table)
plt.xticks(np.linspace(0,11,12,endpoint=True))
plt.xlabel('Month')
plt.ylabel('Average Log Error')
plt.show()

######### AVERAGE ABSOLUTE LOG ERROR PER MONTH #########

abs_log_error_list = train_with_months[['logerror','sale_month']]
abs_log_error_list['logerror'] = np.abs(abs_log_error_list['logerror'])

avg_abs_logs_table = []
for i in range(1,13):
    avg_logs = np.mean(abs_log_error_list[abs_log_error_list['sale_month'] == i]['logerror'])
    avg_abs_logs_table.append(avg_logs)

plt.plot(avg_abs_logs_table)
plt.xticks(np.linspace(0,11,12,endpoint=True))
plt.yticks(np.linspace(0.05,0.09,5,endpoint=True))
plt.xlabel('Month')
plt.ylabel('Average Absolute Log Error')
plt.show()

######### Create New Features #########

property_data['taxpersqft'] = property_data['taxvaluedollarcnt'] / property_data['calculatedfinishedsquarefeet']
property_data['bathpersqft'] = property_data['bathroomcnt'] / property_data['calculatedfinishedsquarefeet']
property_data['roompersqft'] = property_data['roomcnt'] / property_data['calculatedfinishedsquarefeet']
property_data['bedroompersqft'] = property_data['bedroomcnt'] / property_data['calculatedfinishedsquarefeet']

######### Merge the Datasets #########

#train_data.drop(['sale_year','sale_day'],axis=1,inplace=True)
merged_data = train_data.merge(property_data,on='parcelid',how='left')
merged_data.head(5)

train_data.drop(['sale_month','sale_year','sale_day'],axis=1,inplace=True)

######### Draw Geographical Maps #########

# exploratory analysis
# location distribution
plt.figure(figsize=(12,12))
fig=sns.jointplot(x=merged_data.longitude.values, y=merged_data.latitude.values, size=10,color='b')
fig.set_axis_labels('Longitude','Latitude')
#plt.colorbar(orientation="vertical",fraction=0.07)
plt.show()

######### Univariate Analysis #########

# Now let us look at the correlation coefficient of each of these variables #
x_cols = [col for col in merged_data.columns if col not in ['logerror'] if merged_data[col].dtype == 'float64']

labels = []
values = []
for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(merged_data[col].values, merged_data.logerror.values)[0, 1])
corr_df = pd.DataFrame({'col_labels': labels, 'corr_values': values})
corr_df = corr_df.sort_values(by='corr_values')

ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12, 40))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='b')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
# autolabel(rects)
plt.show()

corr_df_sel = corr_df.ix[(corr_df['corr_values']>0.02) | (corr_df['corr_values'] < -0.01)]

######### Important variables correlation map #########

cols_to_use = corr_df_sel.col_labels.tolist()
temp_df = merged_data[cols_to_use]
corrmat = temp_df.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))
# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

import gc

#print('Loading data ...')
#train = pd.read_csv('../input/train_2016.csv')
#prop = pd.read_csv('../input/properties_2016.csv')

for c, dtype in zip(merged_data.columns, merged_data.dtypes):	
    if dtype == np.float64 or dtype == np.int64:		
        merged_data[c] = merged_data[c].astype(np.float32)

#df_train = train_data.merge(property_data, how='left', on='parcelid')
df_train = merged_data


merged_data.drop(['sale_month','sale_year','sale_day'],axis=1,inplace=True)
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 90000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

'''
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.345    # feature_fraction (small values => use very different submodels)
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3
'''

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

print("Prepare for the prediction ...")
sample = pd.read_csv('D:/AI ML/sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(property_data, on='parcelid', how='left')
del sample; gc.collect()
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)

del x_test; gc.collect()

print("Start write result ...")
sub = pd.read_csv('D:/AI ML/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('D:/AI ML/lgb_results.csv', index=False, float_format='%.4f')

